In [1]:
# 최근 영업일 가져오기
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)

import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

print(biz_day)

  |  2025.01.15
20250115


In [2]:
# 업종분류 현황 크롤링
from io import BytesIO
import pandas as pd
import cloudscraper

scraper = cloudscraper.create_scraper()

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'mktId': 'STK',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp_stk = scraper.post(gen_otp_url, gen_otp_stk, headers=headers).text

print(otp_stk)

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = scraper.post(down_url, {'code': otp_stk}, headers=headers)
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')

sector_stk.head()

Nl4gSbNx/ehzaW2VG5KkapUkYhnlvYYU8itw/xakS04RtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eUCgbKcPqzxHxTq/BRX8gTUtBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,일반서비스,4155,-20,-0.48,188025213645
1,006840,AK홀딩스,KOSPI,기타금융,10040,10,0.10,133005512440
2,027410,BGF,KOSPI,기타금융,3455,0,0.00,330701512905
3,282330,BGF리테일,KOSPI,유통,101800,-1000,-0.97,1759501630800
4,138930,BNK금융지주,KOSPI,기타금융,11590,420,3.76,3713861665930


In [3]:
# 코스닥 시장의 데이터도 동일한 과정을 통해 다운로드
gen_otp_ksq = {
    'mktId': 'KSQ',  # 코스닥 입력
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
otp_ksq = scraper.post(gen_otp_url, gen_otp_ksq, headers=headers).text

down_sector_ksq = scraper.post(down_url, {'code': otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

sector_ksq.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,의료·정밀기기,1867,-13,-0.69,99061227680
1,054620,APS,KOSDAQ,금융,4995,-5,-0.10,99371633895
2,265520,AP시스템,KOSDAQ,기계·장비,16970,-220,-1.28,259325714370
3,211270,AP위성,KOSDAQ,운송장비·부품,13220,-110,-0.83,199388058880
4,109960,AP헬스케어,KOSDAQ,유통,505,5,1.00,101398580845


In [4]:
# 코스피 데이터와 코스닥 데이터를 하나로 합친다.
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)
krx_sector['종목명'] = krx_sector['종목명'].str.strip()
krx_sector['기준일'] = biz_day

krx_sector.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,일반서비스,4155,-20,-0.48,188025213645,20250115
1,006840,AK홀딩스,KOSPI,기타금융,10040,10,0.10,133005512440,20250115
2,027410,BGF,KOSPI,기타금융,3455,0,0.00,330701512905,20250115
3,282330,BGF리테일,KOSPI,유통,101800,-1000,-0.97,1759501630800,20250115
4,138930,BNK금융지주,KOSPI,기타금융,11590,420,3.76,3713861665930,20250115


In [5]:
# 개별종목 지표 크롤링
from io import BytesIO
import pandas as pd
import cloudscraper

scraper = cloudscraper.create_scraper()

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType': '1',
    'mktId': 'ALL',
    'trdDd': biz_day,
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp = scraper.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = scraper.post(down_url, {'code': otp}, headers=headers)

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
krx_ind['기준일'] = biz_day

krx_ind.head()

,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률,기준일
0,060310,3S,1867,-13,-0.69,54.0,34.57,NaN,NaN,998.0,1.87,0,0.00,20250115
1,095570,AJ네트웍스,4155,-20,-0.48,367.0,11.32,1066.0,3.90,9326.0,0.45,270,6.50,20250115
2,006840,AK홀딩스,10040,10,0.10,2635.0,3.81,NaN,NaN,44339.0,0.23,200,1.99,20250115
3,054620,APS,4995,-5,-0.10,667.0,7.49,NaN,NaN,11683.0,0.43,0,0.00,20250115
4,265520,AP시스템,16970,-220,-1.28,3997.0,4.25,4607.0,3.68,21396.0,0.79,270,1.59,20250115


In [6]:
# 데이터 정리하기
# 먼저 두 데이터에 공통으로 존재하지 않는 종목, 즉 하나의 데이터에만 존재하는 종목을 살펴보도록 하자.
diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))
print(diff)

['코오롱티슈진', 'NH올원리츠', '이지스밸류리츠', '신한알파리츠', '헝셩그룹', '크리스탈신소재', '엘브이엠씨홀딩스', '롯데리츠', '이리츠코크렙', '맵스리얼티1', 'SBI핀테크솔루션즈', '오가닉티코스메틱', '로스웰', '소마젠', '애머릿지', 'SK리츠', '씨엑스아이', '맥쿼리인프라', '코람코라이프인프라리츠', '스타에스엠리츠', '잉글우드랩', '네오이뮨텍', '한국패러랠', 'KB발해인프라', '마스턴프리미어리츠', '디앤디플랫폼리츠', '신한서부티엔디리츠', '이지스레지던스리츠', '에이리츠', '미래에셋글로벌리츠', '이스트아시아홀딩스', '삼성FN리츠', '윙입푸드', '케이탑리츠', 'NH프라임리츠', '컬러레이', '코람코더원리츠', '제이알글로벌리츠', '엑세스바이오', 'GRT', '신한글로벌액티브리츠', '한화리츠', '한국ANKOR유전', 'JTC', '고스트스튜디오', '프레스티지바이오파마', '미래에셋맵스리츠', 'KB스타리츠', '글로벌에스엠', 'ESR켄달스퀘어리츠']


In [7]:
# 두 데이터를 합쳐준다
kor_ticker = pd.merge(krx_sector,
                      krx_ind,
                      on=krx_sector.columns.intersection(
                          krx_ind.columns).tolist(),
                      how='outer')

kor_ticker.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률
0,000020,동화약품,KOSPI,제약,6370,-40,-0.62,177923463900,20250115,991.0,6.43,NaN,NaN,13413.0,0.47,180.0,2.83
1,000040,KR모터스,KOSPI,운송장비·부품,453,-8,-1.74,27240189204,20250115,NaN,NaN,NaN,NaN,618.0,0.73,0.0,0.00
2,000050,경방,KOSPI,유통,6570,-10,-0.15,180118323900,20250115,NaN,NaN,NaN,NaN,29623.0,0.22,125.0,1.90
3,000070,삼양홀딩스,KOSPI,기타금융,62400,-400,-0.64,534410510400,20250115,22269.0,2.80,NaN,NaN,257475.0,0.24,3500.0,5.61
4,000075,삼양홀딩스우,KOSPI,기타금융,50600,-400,-0.78,15385334800,20250115,NaN,NaN,NaN,NaN,NaN,NaN,3550.0,7.02


In [8]:
#  일반적인 종목과 스팩, 우선주, 리츠, 기타 주식을 구분해주도록 한다
print(kor_ticker[kor_ticker['종목명'].str.contains('스팩|제[0-9]+호')]['종목명'].values)

['미래에셋비전스팩1호' '키움제6호스팩' '신한제10호스팩' '엔에이치스팩23호' '케이비제21호스팩' '삼성스팩6호'
 '신영스팩8호' '유안타제9호스팩' '키움제7호스팩' '유안타제10호스팩' '하나금융25호스팩' '에스케이증권제8호스팩'
 '엔에이치스팩24호' '엔에이치스팩25호' '삼성스팩7호' '엔에이치스팩26호' 'IBKS제20호스팩' '교보13호스팩'
 '엔에이치스팩27호' '유진스팩9호' '대신밸런스제14호스팩' 'IBKS제21호스팩' '미래에셋드림스팩1호' '유안타제11호스팩'
 '비엔케이제1호스팩' '신영스팩9호' '유안타제12호스팩' '미래에셋비전스팩2호' '하나26호스팩' '키움제8호스팩'
 '하나27호스팩' '삼성스팩8호' 'IBKS제22호스팩' '미래에셋비전스팩3호' '유안타제13호스팩' '하이제8호스팩'
 '유안타제14호스팩' '엔에이치스팩29호' '상상인제4호스팩' '신한제11호스팩' '하나29호스팩' '하나28호스팩'
 'KB제25호스팩' '한화플러스제4호스팩' '에스케이증권제9호스팩' 'DB금융스팩11호' '교보14호스팩' '대신밸런스제15호스팩'
 '대신밸런스제16호스팩' '에스케이증권제10호스팩' 'KB제26호스팩' '한국제12호스팩' '에이치엠씨제6호스팩'
 '한국제13호스팩' 'KB제27호스팩' '교보15호스팩' '엔에이치스팩30호' 'IBKS제23호스팩' '삼성스팩9호'
 '유진스팩10호' 'IBKS제24호스팩' '하나30호스팩' '하나31호스팩' '대신밸런스제17호스팩' '신영스팩10호'
 '에스케이증권제11호스팩' '에스케이증권제12호스팩' '유안타제15호스팩' '비엔케이제2호스팩' '에스케이증권제13호스팩'
 '유안타제16호스팩' '신한제12호스팩' '신한제13호스팩' '하나32호스팩' '하나33호스팩' 'KB제28호스팩'
 '에이치엠씨제7호스팩' '미래에셋비전스팩4호' '미래에셋비전스팩5호' '한국제14호스팩' 'DB금융스팩12호' '이베스트스팩6호'
 'KB제29호스팩' '미래에셋비전스팩6호' '대신밸런스제1

In [9]:
print(kor_ticker[kor_ticker['종목코드'].str[-1:] != '0']['종목명'].values)

['삼양홀딩스우' '하이트진로2우B' '유한양행우' '하이트진로홀딩스우' '두산우' '두산2우B' 'DL우' '유유제약1우'
 '유유제약2우B' '노루홀딩스우' '흥국화재우' '현대건설우' '삼성화재우' '한화우' '한화3우B' 'CJ우' 'CJ4우(전환)'
 'JW중외제약우' 'JW중외제약2우B' '부국증권우' 'BYC우' 'SK증권우' '동양우' '동양2우B' '대상우' '한양증권우'
 '대한제당우' '코오롱우' '넥센타이어1우B' '진흥기업우B' '진흥기업2우B' '아모레G우' '아모레G3우(전환)' '금호건설우'
 '코오롱글로벌우' '유화증권우' '유안타증권우' '대한항공우' '한화투자증권우' '대신증권우' '대신증권2우B' 'LG우'
 '남양유업우' '태양금속우' 'NPC우' '세방우' '서울식품우' '깨끗한나라우' '덕성우' '성신양회우' '롯데지주우'
 '녹십자홀딩스2우' '롯데칠성우' '현대차우' '현대차2우B' '현대차3우B' '넥센우' '크라운해태홀딩스우' '삼성전자우'
 'NH투자증권우' '동부건설우' 'SK디스커버리우' '대원전선우' '삼성SDI우' '미래에셋증권우' '미래에셋증권2우B'
 '일양약품우' '코리아써우' '코리아써키트2우B' '대덕1우' '남선알미우' '호텔신라우' '삼성전기우' '태영건설우'
 '한화솔루션우' 'S-Oil우' 'CJ씨푸드1우' '금호석유우' '계양전기우' '금강공업우' '동원시스템즈우' '성문전자우'
 '신풍제약우' '대교우B' '대호특수강우' '삼성물산우B' '소프트센우' 'SK우' '해성산업1우' 'LG생활건강우' 'LG화학우'
 'LG전자우' '한국금융지주우' 'GS우' '대상홀딩스우' '노루페인트우' '아모레퍼시픽우' 'SK이노베이션우' 'CJ제일제당 우'
 'LX하우시스우' '코오롱인더우' '삼양사우' '한진칼우' '크라운제과우' 'SK케미칼우' '두산퓨얼셀1우' '두산퓨얼셀2우B'
 '솔루스첨단소재1우' '솔루스첨단소재2우B' '대덕전자1우' '티와이홀딩스우' 'DL이앤씨우' 'DL이앤씨2우(전환

In [10]:
print(kor_ticker[kor_ticker['종목명'].str.endswith('리츠')]['종목명'].values)

['에이리츠' '케이탑리츠' '스타에스엠리츠' '신한알파리츠' '롯데리츠' '이지스밸류리츠' 'NH프라임리츠' '제이알글로벌리츠'
 '이지스레지던스리츠' '코람코라이프인프라리츠' '미래에셋맵스리츠' '마스턴프리미어리츠' 'ESR켄달스퀘어리츠' '디앤디플랫폼리츠'
 'SK리츠' '미래에셋글로벌리츠' 'NH올원리츠' '신한서부티엔디리츠' '코람코더원리츠' 'KB스타리츠' '삼성FN리츠'
 '한화리츠' '신한글로벌액티브리츠']


In [11]:
# 해당 종목들을 구분하여 표기해주도록 한다
import numpy as np

kor_ticker['종목구분'] = np.where(kor_ticker['종목명'].str.contains('스팩|제[0-9]+호'), '스팩',
                              np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                                       np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠',
                                                np.where(kor_ticker['종목명'].isin(diff),  '기타',
                                                '보통주'))))
kor_ticker = kor_ticker.reset_index(drop=True)
kor_ticker.columns = kor_ticker.columns.str.replace(' ', '')
kor_ticker = kor_ticker[['종목코드', '종목명', '시장구분', '종가',
                         '시가총액', '기준일', 'EPS', '선행EPS', 'BPS', '주당배당금', '종목구분']]
kor_ticker = kor_ticker.replace({np.nan: None})
kor_ticker['기준일'] = pd.to_datetime(kor_ticker['기준일'])

kor_ticker.head()

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분
0,000020,동화약품,KOSPI,6370,177923463900,2025-01-15,991.0,None,13413.0,180.0,보통주
1,000040,KR모터스,KOSPI,453,27240189204,2025-01-15,None,None,618.0,0.0,보통주
2,000050,경방,KOSPI,6570,180118323900,2025-01-15,None,None,29623.0,125.0,보통주
3,000070,삼양홀딩스,KOSPI,62400,534410510400,2025-01-15,22269.0,None,257475.0,3500.0,보통주
4,000075,삼양홀딩스우,KOSPI,50600,15385334800,2025-01-15,None,None,None,3550.0,우선주


In [15]:
# 해당 정보를 DB에 저장한다
import pymysql

con = pymysql.connect(user='root',
                      passwd='1234',
                      host='127.0.0.1',
                      db='stock_db',
                      charset='utf8')

mycursor = con.cursor()
query = f"""
    insert into kor_ticker (종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분)
    values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) as new
    on duplicate key update
    종목명=new.종목명,시장구분=new.시장구분,종가=new.종가,시가총액=new.시가총액,EPS=new.EPS,선행EPS=new.선행EPS,
    BPS=new.BPS,주당배당금=new.주당배당금,종목구분 = new.종목구분;
"""

args = kor_ticker.values.tolist()

mycursor.executemany(query, args)
con.commit()

con.close()

"db 저장 성공"

'db 저장 성공'